In [1]:
import pandas as pd

In [2]:
import geopandas as gpd
eq = gpd.read_file('data/boundaries.geojson')

In [3]:
n = []
for ii, row in eq.iterrows():
    if ('Dublin' in row['ENG_NAME_VALUE']) or ('Laoghaire' in row['ENG_NAME_VALUE']):
        n.append(ii)

In [4]:
eq = eq.iloc[n].reset_index(drop=True)

In [5]:
eq = eq.sort_values(by='ENG_NAME_VALUE').reset_index(drop=True)

eq.to_file('data/boundaries.geojson', driver="GeoJSON")

In [6]:
from shapely.geometry import shape

tst = eq[0:3][['ENG_NAME_VALUE','geometry']]



In [7]:
zones = eq.dissolve(by='ENG_NAME_VALUE').reset_index(drop=False)

In [8]:
zones['ENG_NAME_VALUE']=zones['ENG_NAME_VALUE'].apply(lambda x: x.split('(')[0])

In [9]:
zones = zones[['ENG_NAME_VALUE','geometry']]

In [10]:
zones['ENG_NAME_VALUE']=zones['ENG_NAME_VALUE'].apply(lambda x: x.lower().strip().replace(' ','-'))

In [11]:
files = dir('.csv')

In [12]:
import glob

files = glob.glob( '*.csv' )

In [13]:
tmp = pd.read_csv('data/dublin-bay-south-transportation.csv')
tmp = tmp[tmp['travel_bounds']=='TOTAL']
ts = dict()
for ii, row in tmp.iterrows():
    for column in tmp.columns[2:7]:
        ts[row['mode']+'_'+column]=[]


In [14]:
for ii, row in zones.iterrows():
    f = [item for item in files if row['ENG_NAME_VALUE'] in item]
    if len(f)==1:
        tmp = pd.read_csv('data/'+f[0])
    else:
        print(row['ENG_NAME_VALUE'])
        tmp = pd.read_csv('data/dublin-dún-laoghaire-transportation.csv')
    
    tmp = tmp[tmp['travel_bounds']=='TOTAL']
    for ii, row in tmp.iterrows():
        for column in tmp.columns[2:7]:
            ts[row['mode']+'_'+column].append(row[column])

    if 'TRAM' not in list(tmp['mode']):
        for column in tmp.columns[2:7]:
                ts['TRAM'+'_'+column].append(0)

    if 'RAIL' not in list(tmp['mode']):
        for column in tmp.columns[2:7]:
                ts['RAIL'+'_'+column].append(0)



   

dublin-bay-north
dublin-bay-south
dublin-central
dublin-fingal-east
dublin-fingal-west
dublin-mid-west
dublin-north-west
dublin-rathdown
dublin-south-central
dublin-south-west
dublin-west
dún-laoghaire


In [15]:
dfnew = zones.join(pd.DataFrame(ts))

In [16]:
dfnew['TOTAL_trips']=dfnew['AUTOMOBILE_trips']+dfnew['CYCLING_trips']+dfnew['BUS_trips']+dfnew['ON FOOT_trips']+dfnew['RAIL_trips']+dfnew['TRAM_trips']

In [17]:
dfnew['walk_pct']=dfnew['ON FOOT_trips']/dfnew['TOTAL_trips']
dfnew['cycle_pct']=dfnew['CYCLING_trips']/dfnew['TOTAL_trips']

In [18]:
dfnew['active_pct']=dfnew['walk_pct']+dfnew['cycle_pct']

In [19]:
dfother = gpd.read_file('/Users/rudi/Desktop/active-travel/general_data/census_data.geojson')

In [20]:
dfnew=dfnew.rename({'cycle_pct':'Bicycle - Total_pct','walk_pct':'On foot - Total_pct','active_pct':'Active travel - Total_pct'},axis=1)

In [21]:
test = pd.concat([dfother,dfnew])

In [22]:
test['ID']=range(len(test))

In [23]:
b = []
for jj, row2 in dfother.iterrows():
    a = ''
    for ii,row in dfnew.iterrows():
        if row2['geometry'].within(row['geometry']):
            a = row['ENG_NAME_VALUE']
            break
    b.append(a)



In [24]:
bbox = []
for ii, row in dfnew.iterrows():
    ss = str(dfnew.envelope.iloc[ii])
    cs = []
    for item in ss.split(','):
        item2 = item.split(' ')
        for item in item2:
            item = item.replace('(','')
            item = item.replace(')','')
            if len(item)>0 and item!='POLYGON':
                cs.append(item)

    cs = list(dict.fromkeys(cs))
    cs.sort()
    item = cs[1]+', '+ cs[2]+', '+ cs[0]+', '+ cs[3]
    bbox.append(item)

dfnew['bbox']=bbox

In [25]:
dfnew=dfnew.rename({'Bicycle - Total_pct':'cycle_pct','On foot - Total_pct':'walk_pct','Active travel - Total_pct':'active_pct'},axis=1)

In [26]:
a = dfnew.sum()

/var/folders/c6/jsjxnj0j6vqbnpqy37npy5dw0000gn/T/ipykernel_42095/1641076267.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  a = dfnew.sum()


In [27]:
sums = pd.DataFrame(a).reset_index(drop=False)

In [28]:
pd.DataFrame(a).reset_index(drop=False).to_excel('data/summs.xlsx')

In [29]:
dfa = pd.DataFrame(dfnew['ENG_NAME_VALUE'])

In [30]:
dfa['nam']=dfa['ENG_NAME_VALUE'].apply(lambda x: x.split('-'))

In [31]:
n = []
for ii, row in dfa.iterrows():
    s = ''
    for item in row['nam']:
        s = s+item.capitalize() + ' '
    s = s.strip()
    n.append(s)

In [32]:
dfa['n']=n

In [33]:
sums = sums.T

In [34]:
sums.rename(columns=sums.iloc[0],inplace=True)
sums.drop(sums.index[0],inplace=True)


In [35]:
sums.drop(['bbox','ENG_NAME_VALUE'],axis=1,inplace=True)

In [36]:
sums['walk_pct'].iloc[0]=sums['ON FOOT_trips'].iloc[0]/sums['TOTAL_trips'].iloc[0]
sums['cycle_pct'].iloc[0]=sums['CYCLING_trips'].iloc[0]/sums['TOTAL_trips'].iloc[0]
sums['active_pct'].iloc[0]=(sums['ON FOOT_trips'].iloc[0]+sums['CYCLING_trips'].iloc[0])/sums['TOTAL_trips'].iloc[0]

In [37]:
sums

,AUTOMOBILE_trips,AUTOMOBILE_full_distance_km,AUTOMOBILE_gpc_distance_km,AUTOMOBILE_full_co2e_tons,AUTOMOBILE_gpc_co2e_tons,BUS_trips,BUS_full_distance_km,BUS_gpc_distance_km,BUS_full_co2e_tons,BUS_gpc_co2e_tons,...,RAIL_gpc_co2e_tons,TRAM_trips,TRAM_full_distance_km,TRAM_gpc_distance_km,TRAM_full_co2e_tons,TRAM_gpc_co2e_tons,TOTAL_trips,walk_pct,cycle_pct,active_pct
0,398218992,5055245184,2812140438.0,918791.3016,511106.7972,33159888,289523520,162596292.0,186764.976,104887.1352,...,0.0,8472828,100331292,50165646.0,0.0,0.0,708792372,0.298191,0.031147,0.329338


In [38]:
dfnew.to_file('data/divisions.geojson', driver="GeoJSON")

In [39]:
dfnew['PUBLIC_trips']=dfnew['BUS_trips']+dfnew['RAIL_trips']+dfnew['TRAM_trips']
dfnew['PUBLIC_full_distance_km']=dfnew['BUS_full_distance_km']+dfnew['RAIL_full_distance_km']+dfnew['TRAM_full_distance_km']
dfnew['PUBLIC_gpc_distance_km']=dfnew['BUS_gpc_distance_km']+dfnew['RAIL_gpc_distance_km']+dfnew['TRAM_gpc_distance_km']
dfnew['PUBLIC_full_co2e_tons']=dfnew['BUS_full_co2e_tons']+dfnew['RAIL_full_co2e_tons']+dfnew['TRAM_full_co2e_tons']
dfnew['PUBLIC_gpc_co2e_tons']=dfnew['BUS_gpc_co2e_tons']+dfnew['RAIL_gpc_co2e_tons']+dfnew['TRAM_gpc_co2e_tons']


In [40]:
droplist = [item for item in dfnew.columns if 'TRAM' in item or 'RAIL' in item or 'BUS' in item]

In [41]:
dfnew.drop(droplist,axis=1,inplace=True)

In [42]:
trips = [item for item in dfnew.columns if 'trip' in item]

In [43]:
dfnew.to_file('data/districts.geojson', driver="GeoJSON")  
